In [1]:
# importing libraries
using CSV, DataFrames, JuMP, Gurobi;

In [2]:
# getting data
distances = CSV.read("distances.csv", DataFrame, header=false)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,536.92,1095.01,341.398,649.241,269.852,333.921,585.559,3571.18
2,536.92,0.0,1503.25,820.251,1179.31,752.758,867.362,730.888,4107.96
3,1095.01,1503.25,0.0,756.166,965.459,826.045,858.04,845.624,2979.54
4,341.398,820.251,756.166,0.0,534.611,72.1738,188.874,484.196,3335.06
5,649.241,1179.31,965.459,534.611,0.0,551.455,365.379,1018.7,2936.02
6,269.852,752.758,826.045,72.1738,551.455,0.0,188.056,479.385,3389.51
7,333.921,867.362,858.04,188.874,365.379,188.056,0.0,665.014,3245.17
8,585.559,730.888,845.624,484.196,1018.7,479.385,665.014,0.0,3715.26
9,3571.18,4107.96,2979.54,3335.06,2936.02,3389.51,3245.17,3715.26,0.0


In [3]:
# rankings, get last column
rankings = CSV.read("rankings.csv", DataFrame, header=false)[:,2]
@show rankings

rankings = [64, 39, 10, 44, 12, 42, 46, 59, 8, 57, 24, 25, 30, 27, 29, 52, 7, 55, 28, 36, 53, 18, 2, 49, 21, 9, 1, 20, 34, 33, 22, 63, 40, 38, 58, 56, 19, 15, 48, 32, 13, 5, 6, 43, 47, 60, 26, 45, 35, 50, 51, 41, 31, 17, 11, 61, 54, 14, 37, 16, 3, 23, 4, 62]


64-element Vector{Int64}:
 64
 39
 10
 44
 12
 42
 46
 59
  8
 57
 24
 25
 30
  ⋮
 31
 17
 11
 61
 54
 14
 37
 16
  3
 23
  4
 62

In [4]:
distances = Matrix(distances)
rankings = Vector(rankings)
colleges_iter = 1:64

1:64

In [5]:
"""
We compute normalized ranking scores as
1 - (team rank) / (total number of teams)
"""

ranking_scores = 1 .- rankings ./ 64;

"""
We compute normalized distance scores as
1 - (distance) / (maximum distance)
"""

distance_scores = 1 .- distances ./ maximum(distances);

In [6]:
# creating model and decision variables
model = Model(Gurobi.Optimizer)


# s_ij = 1 if college i hosts college j, 0 otherwise
@variable(model, s[colleges_iter, colleges_iter], Bin)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-19


2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, 1:64
    Dimension 2, 1:64
And data, a 64×64 Matrix{VariableRef}:
 s[1,1]   s[1,2]   s[1,3]   s[1,4]   s[1,5]   …  s[1,62]   s[1,63]   s[1,64]
 s[2,1]   s[2,2]   s[2,3]   s[2,4]   s[2,5]      s[2,62]   s[2,63]   s[2,64]
 s[3,1]   s[3,2]   s[3,3]   s[3,4]   s[3,5]      s[3,62]   s[3,63]   s[3,64]
 s[4,1]   s[4,2]   s[4,3]   s[4,4]   s[4,5]      s[4,62]   s[4,63]   s[4,64]
 s[5,1]   s[5,2]   s[5,3]   s[5,4]   s[5,5]      s[5,62]   s[5,63]   s[5,64]
 s[6,1]   s[6,2]   s[6,3]   s[6,4]   s[6,5]   …  s[6,62]   s[6,63]   s[6,64]
 s[7,1]   s[7,2]   s[7,3]   s[7,4]   s[7,5]      s[7,62]   s[7,63]   s[7,64]
 s[8,1]   s[8,2]   s[8,3]   s[8,4]   s[8,5]      s[8,62]   s[8,63]   s[8,64]
 s[9,1]   s[9,2]   s[9,3]   s[9,4]   s[9,5]      s[9,62]   s[9,63]   s[9,64]
 s[10,1]  s[10,2]  s[10,3]  s[10,4]  s[10,5]     s[10,62]  s[10,63]  s[10,64]
 s[11,1]  s[11,2]  s[11,3]  s[11,4]  s[11,5]  …  s[11,62]  s[11,63]  s[11,64]
 s[

In [7]:
set_optimizer_attribute(model, "OutputFlag", 0)

In [8]:
# defining objective function

# weight we give to rankings
RW = 0.5
# weight we give to distances
DW = 0.5

@objective(model, Max, RW * sum(ranking_scores[i] * s[i,i] for i in colleges_iter) + DW * sum(distance_scores[i,j] * s[i,j] for i in colleges_iter, j in colleges_iter))

0.6953125 s[2,2] + 0.921875 s[3,3] + 0.65625 s[4,4] + 0.90625 s[5,5] + 0.671875 s[6,6] + 0.640625 s[7,7] + 0.5390625 s[8,8] + 0.9375 s[9,9] + 0.5546875 s[10,10] + 0.8125 s[11,11] + 0.8046875 s[12,12] + 0.765625 s[13,13] + 0.7890625 s[14,14] + 0.7734375 s[15,15] + 0.59375 s[16,16] + 0.9453125 s[17,17] + 0.5703125 s[18,18] + 0.78125 s[19,19] + 0.71875 s[20,20] + 0.5859375 s[21,21] + 0.859375 s[22,22] + 0.984375 s[23,23] + 0.6171875 s[24,24] + 0.8359375 s[25,25] + 0.9296875 s[26,26] + 0.9921875 s[27,27] + 0.84375 s[28,28] + 0.734375 s[29,29] + 0.7421875 s[30,30] + 0.828125 s[31,31] + 0.5078125 s[32,32] + 0.6875 s[33,33] + 0.703125 s[34,34] + 0.546875 s[35,35] + 0.5625 s[36,36] + 0.8515625 s[37,37] + 0.8828125 s[38,38] + 0.625 s[39,39] + 0.75 s[40,40] + 0.8984375 s[41,41] + 0.9609375 s[42,42] + 0.953125 s[43,43] + 0.6640625 s[44,44] + 0.6328125 s[45,45] + 0.53125 s[46,46] + 0.796875 s[47,47] + 0.6484375 s[48,48] + 0.7265625 s[49,49] + 0.609375 s[50,50] + 0.6015625 s[51,51] + 0.6796875 s[52

In [9]:
# defining constraints

# each college can only go to one host
@constraint(model, [j in colleges_iter], sum(s[i,j] for i in colleges_iter) == 1)

# there are 8 host colleges
@constraint(model, sum(s[i,i] for i in colleges_iter) == 8)

# each hosting college hosts 8 colleges, including itself (only if s[i,i] = 1)
@constraint(model, [i in colleges_iter], sum(s[i,j] for j in colleges_iter) == 8 * s[i,i])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:64
And data, a 64-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 -7 s[1,1] + s[1,2] + s[1,3] + s[1,4] + s[1,5] + s[1,6] + s[1,7] + s[1,8] + s[1,9] + s[1,10] + s[1,11] + s[1,12] + s[1,13] + s[1,14] + s[1,15] + s[1,16] + s[1,17] + s[1,18] + s[1,19] + s[1,20] + s[1,21] + s[1,22] + s[1,23] + s[1,24] + s[1,25] + s[1,26] + s[1,27] + s[1,28] + s[1,29] + s[1,30] + s[1,31] + s[1,32] + s[1,33] + s[1,34] + s[1,35] + s[1,36] + s[1,37] + s[1,38] + s[1,39] + s[1,40] + s[1,41] + s[1,42] + s[1,43] + s[1,44] + s[1,45] + s[1,46] + s[1,47] + s[1,48] + s[1,49] + s[1,50] + s[1,51] + s[1,52] + s[1,53] + s[1,54] + s[1,55] + s[1,56] + s[1,57] + s[1,58] + s[1,59] + s[1,60

In [10]:
optimize!(model)

@show objective_value(model)

objective_value(model) = 33.26719908582976


33.26719908582976

In [11]:
# printing host cities
for i in colleges_iter
    if value(s[i,i]) == 1
        println("College ", i, " is a host city.")
    end
end

College 3 is a host city.
College 26 is a host city.
College 27 is a host city.
College 42 is a host city.
College 43 is a host city.
College 55 is a host city.
College 60 is a host city.
College 63 is a host city.


In [12]:
# now we test various values of RW and DW to see how the objective value changes and which cities are picked
function test_RW_DW(RW, DW)
    @objective(model, Max, RW * sum(ranking_scores[i] * s[i,i] for i in colleges_iter) + DW * sum(distance_scores[i,j] * s[i,j] for i in colleges_iter, j in colleges_iter))
    optimize!(model)
    println("RW = ", RW, ", DW = ", DW, ", objective value = ", objective_value(model))
    # ranking score
    println("Ranking score = ", sum(ranking_scores[i] * value(s[i,i]) for i in colleges_iter))
    # distance score
    println("Distance score = ", sum(distance_scores[i,j] * value(s[i,j]) for i in colleges_iter, j in colleges_iter))
    for i in colleges_iter
        if value(s[i,i]) == 1
            println("College ", i, " is a host city.")
        end
    end
end

test_RW_DW (generic function with 1 method)

In [13]:
for RW in 0:0.1:1
    DW = 1 - RW
    test_RW_DW(RW, DW)
end

RW = 0.0, DW = 1.0, objective value = 59.726686278773286
Ranking score = 4.4375
Distance score = 59.72668627877326
College 12 is a host city.
College 19 is a host city.
College 22 is a host city.
College 25 is a host city.
College 29 is a host city.
College 39 is a host city.
College 50 is a host city.
College 63 is a host city.
RW = 0.1, DW = 0.9, objective value = 54.342092273114545
Ranking score = 6.5625
Distance score = 59.65093585901615
College 3 is a host city.
College 22 is a host city.
College 26 is a host city.
College 27 is a host city.
College 30 is a host city.
College 43 is a host city.
College 55 is a host city.
College 63 is a host city.
RW = 0.2, DW = 0.8, objective value = 49.03324868721294
Ranking score = 6.5625
Distance score = 59.65093585901615
College 3 is a host city.
College 22 is a host city.
College 26 is a host city.
College 27 is a host city.
College 30 is a host city.
College 43 is a host city.
College 55 is a host city.
College 63 is a host city.
RW = 0.3, 